<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.5%20Herramientas%20para%20procesamiento%20y%20explotaci%C3%B3n%20de%20Big%20Data/4.5.2%20DataFrames%20de%20Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fundamentos

Los Spark DataFrames son la piedra angular y la forma principal de trabajar con Spark y Python después de Spark 2.0.

Los DataFrames actúan como potentes facilitadores para el manejo de tablas, y manejan fácilmente grandes conjuntos de datos.

El cambio a DataFrames ofrece muchas ventajas:
* Una sintaxis mucho más simple
* Capacidad de usar SQL directamente en el marco de datos
* Las operaciones se distribuyen automáticamente entre los RDD

Si ha usado anteriormente la biblioteca de pandas de Python o incluso R, probablemente ya esté familiarizado con el concepto de DataFrames.

Los Spark DataFrames amplían muchos de estos conceptos, lo que le permite transferir ese conocimiento fácilmente al comprender la sintáxis simple.

Recuerde que la principal ventaja de usar Spark DataFrames frente a otras herramientas es que Spark puede manejar grandes conjuntos de datos que no cabrían en una sola computadora.

In [ ]:
dbutils.fs.ls("/")

In [ ]:
generation = "mod4gen<x>"

Para montar un Blob Storage en Azure Databricks:
```
dbutils.fs.mount(
  source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net",
  mount_point = "/mnt/<mount-name>",
  extra_configs = {"fs.azure.account.key.<storage-account-name>.blob.core.windows.net": "<your-key>"})
```

In [ ]:
dbutils.fs.mount(
  source = "wasbs://input@minidatalake.blob.core.windows.net",
  mount_point = f"/mnt/{generation}/input",
  extra_configs = {"fs.azure.account.key.minidatalake.blob.core.windows.net": "<your-key>"})

In [ ]:
dbutils.fs.mount(
  source = "wasbs://output@minidatalake.blob.core.windows.net",
  mount_point = f"/mnt/{generation}/output",
  extra_configs = {"fs.azure.account.key.minidatalake.blob.core.windows.net": "<your-key>"})

In [ ]:
# dbutils.fs.unmount(f"/mnt/{generation}/input")
# dbutils.fs.unmount(f"/mnt/{generation}/output")

In [ ]:
dbutils.fs.ls(f"/mnt/{generation}/input")

In [ ]:
df = spark.read.csv(f"/mnt/{generation}/input/appl_stock.csv", inferSchema=True, header=True)

In [ ]:
df.show()

In [ ]:
df.display()

Carga de datos semi-estructurados

In [ ]:
df = spark.read.json(f"/mnt/{generation}/input/people.json")

In [ ]:
# Observe cómo faltan datos
display(df)

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.describe().display()

Algunos tipos de datos facilitan la inferencia del esquema (como formatos tabulares como CSV).

Sin embargo, a menudo se tiene que configurar un esquema manualmente si se está trabajando con un método `.read` que no tiene `inferSchema()` incorporado.

Spark tiene algunas funciones necesarias para realizar esta operación, solo se requiere una estructura más específica.

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [ ]:
final_struc = StructType(fields=data_schema)

In [ ]:
df = spark.read.json(f"/mnt/{generation}/input/people.json", schema=final_struc)

In [ ]:
df.printSchema()

In [ ]:
df['age']

In [ ]:
type(df['age'])

In [ ]:
df.select('age')

In [ ]:
type(df.select('age'))

In [ ]:
df.select('age').display()

In [ ]:
# Devuelve la lista de objetos Row
df.head(2)

Múltiples columnas:

In [ ]:
df.select(['age','name'])

In [ ]:
df.select(['age','name']).display()

Creación de nuevas columnas

In [ ]:
# Agregar una nueva columna con una copia simple
df.withColumn('newage', df['age']).display()

In [ ]:
df.display()

In [ ]:
# Renombrado simple
df.withColumnRenamed('age','supernewage').display()

Operaciones más complejas para crear nuevas columnas.

In [ ]:
df.withColumn('doubleage', df['age']*2).display()

In [ ]:
df.withColumn('add_one_age', df['age']+1).display()

In [ ]:
df.withColumn('half_age', df['age']/2).display()

In [ ]:
df.withColumn('half_age',df['age']/2)

### Uso de SQL

Para usar consultas SQL directamente con el dataframe, se debe registrar en una vista temporal

In [ ]:
# Registra el DataFrame como una vista temporal de SQL
df.createOrReplaceTempView("people")

In [ ]:
sql_results = spark.sql("SELECT * FROM people")

In [ ]:
sql_results

In [ ]:
sql_results.display()

In [ ]:
spark.sql("SELECT * FROM people WHERE age=30").display()